In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


/Users/polinab/.virtualenvs/base/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = pd.read_csv('./data/Consumidor_Venta_Producto_UPC_Recom_2018_2020.csv')
english_cols = {'FACTURA_ID': 'INVOICE_ID', 'FACTURA_POSICION_ID': 'INVOICE_POSITION_ID', 'CUSTOMER_ID': 'CUSTOMER_ID',
                'FACTURA_CLASE_DOCUMENTO_ID': 'INVOICE_DOCUMENT_CLASS_ID', 'ANO_MES_FACTURA': 'INVOICE_MONTH_YEAR',
                'ANO_FACTURA': 'ORDER_YEAR', 'MES_FACTURA': 'ORDER_MONTH', 'FECHA_FACTURA': 'INVOICE_DATE',
                'IMP_VENTA_NETO_EUR': 'NET_SALES_EUR', 'CANAL_VENTA_ID': 'SALES_CHANNEL_ID',
                'CANAL_VENTA_DESC': 'SALES_CHANNEL_DESC', 'TEMPORADA_COMERCIAL_ID': 'SEASON_ID',
                'TEMPORADA_COMERCIAL_DESC': 'SEASON_DESC', 'PRODUCTO_ID': 'PRODUCT_ID', 'TALLA': 'SIZE',
                'MATERIAL_ID': 'MATERIAL_ID', 'NUMERO_DEUDOR_PAIS_ID': 'SALE_COUNTRY_ID',
                'NUMERO_DEUDOR_PAIS_DESC': 'SALE_COUNTRY_DESC', 'VENTA_DEVOLUCION': 'SALES_FILTER',
                'JERARQUIA_PROD_ID': 'PRODUCT_HIERARCHY_ID', 'GRUPO_ARTICULO_PRODUCTO_ID': 'PRODUCT_GROUP_ID',
                'GRUPO_ARTICULO': 'PRODUCT_GROUP', 'CONCEPTO': 'CONCEPT', 'LINEA': 'LINE',
                'GENERO_PRODUCTO': 'PRODUCT_GENDER', 'CATEGORIA': 'PRODUCT_CATEGORY', 'TIPOLOGIA': 'PRODUCT_TYPE',
                'COLOR': 'COLOR_INTERN', 'CONSUMER_COLOR': 'COLOR_EXTERN', 'CREMALLERA': 'ZIPPER', 'CORDONES': 'LACES',
                'OUTSOLE_SUELA_TIPO': 'SOLE_TYPE', 'OUTSOLE_SUELA_SUBTIPO': 'SOLE_SUBTYPE',
                'PLANTILLA_EXTRAIBLE': 'REMOVABLE_SOLE', 'CONTACTO_SN': 'PHYSICAL_CONTACT', 'EDAD_SN': 'AGE_AVAILABLE',
                'GENERO_CONTACTO': 'GENDER_AVAILABLE', 'EDAD_COMPRA': 'AGE_AT_PURCHASE',
                'EDAD_RANGO_COMPRA': 'AGE_RANGE',
                'PAIS_CONTACTO': 'COUNTRY_CONTACT_ID', 'PAIS_CONTACTO_DESC': 'COUNTRY_CONTACT_DESC',
                'CIUDAD_CONTACTO': 'CITY_CONTACT', 'IDIOMA_CONTACTO': 'LANGUAGE_CONTACT'}


In [3]:
dropcols = ['INVOICE_MONTH_YEAR', 'ORDER_YEAR','ORDER_MONTH','SALES_CHANNEL_ID',\
            'SALES_CHANNEL_DESC','AGE_AVAILABLE','MATERIAL_ID','COUNTRY_CONTACT_DESC',\
            'SALE_COUNTRY_DESC','SALES_FILTER','SEASON_ID','GENDER_AVAILABLE']

In [4]:
# gets most common country for the code
def country_dict(df, id_field,desc_field):
    country_dict = {x[id_field]:x[desc_field] for x in df.groupby(id_field)[desc_field].apply(lambda x: x.mode()).to_frame().reset_index().to_dict('index').values()}
    return country_dict

In [5]:
country_lookup = country_dict(data, 'PAIS_CONTACTO','PAIS_CONTACTO_DESC')
country_lookup.update(country_dict(data, 'NUMERO_DEUDOR_PAIS_ID','NUMERO_DEUDOR_PAIS_DESC'))
country_lookup



{'AD': 'ANDORRA',
 'AE': 'EMIRATOS ÁRABES UNIDOS',
 'AF': 'AFGANISTÁN',
 'AG': 'ANTIGUA Y BARBUDA',
 'AL': 'ALBANIA',
 'AR': 'ARGENTINA',
 'AT': 'AUSTRIA',
 'AU': 'AUSTRALIA',
 'AZ': 'AZERBAYÁN',
 'BA': 'BOSNIA-HERZEGOVINA',
 'BE': 'BÉLGICA',
 'BG': 'BULGARIA',
 'BR': 'BRAZIL',
 'BY': 'BIELORRUSIA',
 'CA': 'CANADA',
 'CH': 'SWITZERLAND',
 'CL': 'CHILE',
 'CN': 'CHINA',
 'CO': 'COLOMBIA',
 'CR': 'COSTA RICA',
 'CU': 'CUBA',
 'CY': 'CHIPRE',
 'CZ': 'REPÚBLICA CHECA',
 'DE': 'GERMANY',
 'DK': 'DENMARK',
 'EC': 'ECUADOR',
 'EE': 'ESTONIA',
 'EG': 'EGIPTO',
 'ES': 'SPAIN',
 'FI': 'FINLANDIA',
 'FR': 'FRANCE',
 'GB': 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND',
 'GE': 'GEORGIA',
 'GP': 'GUADALUPE',
 'GR': 'GREECE',
 'HK': 'HONG KONG',
 'HR': 'CROACIA',
 'HU': 'HUNGRÍA',
 'IE': 'IRELAND',
 'IL': 'ISRAEL',
 'IN': 'INDIA',
 'IS': 'ISLANDIA',
 'IT': 'ITALY',
 'JP': 'JAPAN',
 'KR': 'KOREA REPUBLIC OF',
 'KZ': 'KAZAJSTÁN',
 'LA': 'LAOS',
 'LI': 'LIECHTENSTEIN',
 'LT': 'LITUANIA',
 'LU':

In [56]:
def preprocess(d):
    d_copy = d.rename(columns=english_cols)
    d_copy['REMOVABLE_SOLE'] = d_copy['REMOVABLE_SOLE'].apply(lambda x: True if x == 'Extraible' else False)
    
    invoice_id_dict = {'ZTON':'Sale','ZDVN':'Return','ZDAN':'Cancelled Return'}
    d_copy['INVOICE_DOCUMENT_CLASS_ID'] = d_copy['INVOICE_DOCUMENT_CLASS_ID'].apply(lambda x: invoice_id_dict[x])
    
    d_copy['INVOICE_DATE'] = pd.to_datetime(d_copy.INVOICE_DATE)
    
    d_copy.AGE_AT_PURCHASE = d_copy.AGE_AT_PURCHASE.replace(0, np.nan)
    
    d_copy.ZIPPER = d_copy.ZIPPER.apply(lambda x: True if x in ('SI','YES') else False)
    d_copy.LACES = d_copy.LACES.apply(lambda x: True if x in ('With laces', 'Con cordones') else False)
    
    product_group_dict = {'Zapatos Adulto':'Adult Shoes',
                          'Bolsos':'Bag',
                          'Bolsos Cartujano':'Bag',
                          'Ropa':'Clothings',
                          'Complementos':'Complements'
                         }
    d_copy.PRODUCT_GROUP = d_copy.PRODUCT_GROUP.apply(lambda x: product_group_dict[x] if x in product_group_dict else x)
    
    color_dict = {'Мульти ассорти':'Multi - Assorted',
                  'красный': 'Red',
                  'розовый' : 'Pink',
                  'желтый' : 'Yellow'
                 }
    d_copy.COLOR_INTERN = d_copy.COLOR_INTERN.apply(lambda x: color_dict[x] if x in color_dict else x)
    
    d_copy.PHYSICAL_CONTACT = d_copy.PHYSICAL_CONTACT.apply(lambda x: True if x == 'S' else False)
    
    
    return d_copy



In [57]:
cleandata = preprocess(data)
cleandata = cleandata.drop(columns=dropcols)



In [61]:
item_fields = ['PRODUCT_ID','PRODUCT_GROUP','CONCEPT','LINE',\
               'PRODUCT_GENDER','PRODUCT_CATEGORY','PRODUCT_TYPE',\
               'LACES','ZIPPER','SOLE_TYPE','SOLE_SUBTYPE','REMOVABLE_SOLE','SEASON_DESC',\
              'COLOR_INTERN']

item_categoricals = ['PRODUCT_GROUP','CONCEPT','LINE','PRODUCT_GENDER',\
                     'PRODUCT_CATEGORY','PRODUCT_TYPE','SOLE_TYPE',\
                     'SOLE_SUBTYPE','SEASON_DESC','COLOR_INTERN'
                    ]


In [64]:

items_attributes = cleandata[item_fields]
pd.get_dummies(items_attributes[item_categoricals])


,PRODUCT_GROUP_Adult Shoes,PRODUCT_GROUP_Bag,PRODUCT_GROUP_Clothings,PRODUCT_GROUP_Complements,PRODUCT_GROUP_Kid's Shoes,PRODUCT_GROUP_Kids,CONCEPT_ABS,CONCEPT_Accesorios,CONCEPT_Accesorios Medwinds,CONCEPT_Adapack,...,COLOR_INTERN_Multi - Assorted,COLOR_INTERN_Navy,COLOR_INTERN_No Color,COLOR_INTERN_Nude,COLOR_INTERN_Pink,COLOR_INTERN_Red,COLOR_INTERN_Rust/Copper,COLOR_INTERN_Turquiose Aqua,COLOR_INTERN_White Natural,COLOR_INTERN_Yellow
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327378,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327379,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327380,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327381,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
